# Werkzeuge zur Darstellung und Modellanpassungen für den Versuch Resonanz

Mit den folgenden Code-Fragmenten zeigen wir Ihnen: 

 * Wie man die Daten des CASSY-Messsystems mit PhyPraKit **einließt**.
 * Wie man die eingelesenen Daten auf ein handhabbares Maß **reduziert**.
 * Wie man die Daten mit Hilfe von [*Spline*](https://de.wikipedia.org/wiki/Spline)-Funktionen **interpoliert**.
 * Wie man die angepassten *Spline*-Funktionen numerisch **differenziert**.  
 * Wie man die eingelesenen Daten mit Hilfe von *matplotlib* **darstellt**.
 * Wie man an die eingelesenen Daten ein **geeignetes Modell anpasst**, um die Parameter $\Omega_{0}$ und $\lambda_{0}$ zu bestimmen.

Alle im Folgenden gezeigten Code-Fragmente lassen sich geeignet kombinieren. 

## Einlesen der Daten des CASSY-Messsystems mit PhyPraKit

Mit dem folgenden Code-Fragment zeigen wir Ihnen:
    
 * Wie man mit der PhyPrakit Funktion *labxParser* Daten aus einer *labx*-Datei des CASSY-Messsystems **einließt**.
 * Wir verwenden hierzu die Beispiel-Datei `CassyExample.labx`, aus dem *tools*-Verzeichnis.

In [105]:
import numpy as np
from PhyPraKit.phyTools import labxParser

# Read data from CASSY; 
# column_names: hosts the column names; 
# data: hosts the individual columns as lists. 
# The argument prlevel steers how much information is printed to screen. We
# set it to 1 here, so that you can learn something about the structure of 
# the labx file. If you want to shut the function up set prlevel=0. Check 
# help(labxParser) to learn more about this function
column_names, data = labxParser("CassyExample.labx", prlevel=1)
# For this example we collect the oservables "Zeit" and "Winkel"
t=[]; phi=[]
# We cut off the first 400 and last 450 lines to reduce the amount of data 
# points from ~1300 to <500. Downsampling is not really an option for this 
# example, since the oscillation period is very small
LOWER_CUT= 400
UPPER_CUT=-450
for i, tag in enumerate(column_names):
    # Column names are encoded in a more cryptic way than necessary by Leybold
    # experts :-(; uncomment the following line to check this encoding
    #print(tag.split(":"))
    tcn = tag.split(":")[1]
    if tcn == "Zeit":
        # Copy full column in variable t
        t = np.array(data[i][LOWER_CUT:UPPER_CUT])
    if tcn == "Winkel":
        # Copy full column in variable phi
        phi = np.array(data[i][LOWER_CUT:UPPER_CUT])

ModuleNotFoundError: No module named 'PhyPraKit.phyTools'

## Dastellung der eingelesenen Daten

Mit dem folgenden Code-Fragment zeigen wir Ihnen:
    
 * Wie man sich mit Hilfe von *numpy* die (numerische) Ableitung $\partial_{t}\varphi$ verschafft.
 * Wie man $\varphi(t)$, $\dot{\varphi}(t)$ und ein Phasenraumportrait $(\varphi, \dot{\varphi})(t)$ mit Hilfe von *matplotlib* darstellt.

In [ ]:
# Choose difference between first an second entry in t as estimate for dt
dt   = t[1]-t[0]
# Apply numerical derivative to phi
dphi = np.gradient(phi, dt)

In [ ]:
import matplotlib.pyplot as plt

# Display phi and dphi
fig = plt.figure("Winkel und Winkelgeschwindigkeit", figsize=(12.0, 12.0))
ax1 = fig.add_subplot(2, 1, 1)
ax1.set_xlabel("t (s)")
ax1.set_ylabel(r"$\varphi(t)$ (rad)")
ax1.plot(t, phi , color="darkblue")
ax1.grid()
ax2 = fig.add_subplot(2, 1, 2)
ax2.set_xlabel("t (s)")
ax2.set_ylabel(r"$\dot{\varphi}(t)$ (rad/s)")
ax2.plot(t, dphi , color="darkcyan")
ax2.grid()

plt.show()

In [ ]:
from scipy import interpolate

# Do a nice cubic spline interpolation with help of the scipy function 
# interpolate
cs_phi  = interpolate.UnivariateSpline(t, phi, s=0)
# Get dphi as derivative of the spline approximation to phi
cs_dphi = cs_phi.derivative()

# Display Phasenraumprotrait
tplt = np.linspace(t[0], t[-1], 50000)
fig = plt.figure("Phasenraumportrait", figsize=(6.0, 6.0))
plt.xlabel(r"$\varphi(t)$ (rad)")
plt.ylabel(r"$\dot{\varphi}(t)$ (rad/s)")
plt.plot(cs_phi(tplt), cs_dphi(tplt) , color="darkblue")
plt.grid()

plt.show()

## Anpassung eines geeigneten Modells an die Daten

Mit dem folgenden Code-Fragment zeigen wir Ihnen:
    
 * Wie man ein geeignetes Modell an die extrahierten Daten anpasst.

In [ ]:
from kafe2 import XYContainer

# Fill XYContainer
xy_data = XYContainer(x_data=t, y_data=phi)
# We add some typical uncertaintes here
xy_data.add_error(axis="x", err_val=0.0125)
xy_data.add_error(axis="y", err_val=0.0500)
xy_data.label = "Data"

from kafe2 import Fit, Plot, ContoursProfiler

# Fit model, we start of with a damped oscillation, and potentially extend by 
# a linear extra term to additional frictional effects of the wheel. The POIs 
# (parameters of interest) are lambda0 and omega0.
def harmonic_damped(x, lambda0=0.01, x0=3.5, omega0=3.1, phi0=0., a=-1.0):
      return x0*np.exp(-x*lambda0)*np.cos(omega0*x+phi0)+a*x
# Declare and do the fit with a bit of cosmetic customization
fit = Fit(xy_data, model_function=harmonic_damped)
fit.assign_model_function_latex_name(r"\Omega_{X}")
fit.assign_parameter_latex_names(
    lambda0 = r"\lambda_{0}", 
    x0      = r"x_{0}", 
    omega0  = r"\omega_{0}",
    phi0    = r"\phi_{0}",
    a       = r"a"
)
# Set limits for parameters if needed. 
fit.limit_parameter("phi0", lower=-180., upper=180.)
fit.limit_parameter("x0", lower=0.)
# Do the fit
fit.do_fit()
# Return a reasonably clean report
fit.report(show_data=False, show_model=False)

# Plot the result with a bit of customization
plot = Plot(fit_objects=fit)
plot.customize('model_line', 'label', [(0, r'Model Pohls Wheel ($\Omega_{X}$)')])
plot.customize('model_line', 'color', [(0, 'darkblue')])
plot.customize('model_line', 'linestyle', [(0, 'solid')])
plot.customize('model_error_band', 'label', [(0, r'$\pm1\sigma$')])
plot.customize('model_error_band', 'color', [(0, 'lightsteelblue')])
plot.x_label = r'$t\,(\mathrm{s})$'
plot.y_label = r'$\varphi\,(rad)$'
plot.plot()  

# Create contour plots, this step should only be used when using less than 300 
# data points, since it may take really long otherwise
#cpf = ContoursProfiler(fit)
#cpf.plot_profiles_contours_matrix()

# Show plot in notebook
plot.show()